In [1]:
import gdown
import os
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import keras
from keras.models import load_model, Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

In [2]:
##**************************
##****** elavation**********
##**************************
def plot_performance(hist,epoch, title, save='/content/',Legs=None):
    keyss=list(hist.history.keys())
    if Legs:
      legs=Legs
    else:
      legs=keyss


    colors= ['red','blue','green','#FFA500']



    fig,(ax1,ax2) = plt.subplots(1,2, figsize = (12,6))
    for i in range(0,len(keyss),2):
      histor = np.array(hist.history[keyss[i+1]])
      ax1.plot(range(1,epoch+1),histor*100, color=colors[i//2], label=legs[i+1])#


    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)')

    ax1.set_ylim((0,100.5))
    ax1.set_xlim((1,epoch))
    ax1.grid(axis='x', color='0.85')
    ax1.grid(axis='y', color='0.85')
    ax1.set_title(title, fontsize=14, fontweight='bold')



    for i in range(0,len(keyss),2):
      histor = hist.history[keyss[i]]
      ax2.plot(range(1,epoch+1),histor, color=colors[i//2], label=legs[i])#

    # Add a legend to the plot


    # Set the axis labels and title
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss',color='red')
    ax2.tick_params(axis='y', colors='red')
    ax2.grid(axis='x', color='0.85')
    ax2.grid(axis='y', color='0.85')

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()

    ax1.legend(lines1 , labels1 , loc='right')
    ax2.legend(lines2, labels2, loc='right')
    plt.savefig(save+'.png', dpi=800)

    # convert the history.history dict to a pandas DataFrame:
    hist_df = pd.DataFrame(hist.history)

    # or save to csv:
    hist_csv_file = save+'-history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

    fig.tight_layout()
    plt.show()
    if save:
      fig.savefig(save+'-'+title+'.jpg',dpi=300)
    plt.show()


#%% Defined Confusion Matrix plot function
def PlotConfusionMatrix(y_test ,y_pred_probs, title='Confusion Matrix', figsize=(8,8), save='/content/'):

    cf_matrix=confusion_matrix(np.argmax(y_test,axis = 1), np.argmax(y_pred_probs,axis = 1))
    # DetaFrame_cm = pdNew.DataFrame(array, range(6), range(5))
    fig = plt.figure(figsize=figsize)
    ax2 = plt.axes()
    ax2.set_title(title)
    ax2.xaxis.tick_top()
    ConfusionMatrixDisplay(cf_matrix).plot(ax=ax2,cmap='Blues',colorbar=False)

    if save:
      fig.savefig(save+'-'+title+'.jpg', dpi=300)
    plt.show()


# claculate model scores
def score_model(y_test, y_pred_oh, y_pred_probs, save ='/content/'):

    acc = accuracy_score(y_test, y_pred_oh)
    prec = precision_score(y_test, y_pred_oh, average = 'weighted')
    rec = recall_score(y_test, y_pred_oh, average = 'weighted')
    fscore = f1_score(np.argmax(y_test,axis = 1), np.argmax(y_pred_oh,axis = 1), average='weighted')
    cmatrix = confusion_matrix(np.argmax(y_test,axis = 1), np.argmax(y_pred_oh,axis = 1))
    # _, conf = pred_confidence(y_pred_probs)

    scores = {'accuracy' : acc,
              'precision' : prec,
              'recall' : rec,
              'F1score': fscore,
              # 'Confidence' : conf,
              'Confusion': cmatrix
              }

    if save:
        import csv
        with open(save+'-Scores','w') as f:
            for key in scores.keys():
                f.write("%s,%s\n"%(key,scores[key]))
    del scores['Confusion']
    return scores

In [3]:
def DataSplit(DataTensor,TempDimLen):
  nclass=DataTensor.shape[0]

  for idx in range(nclass):
      respmat2=DataTensor[idx].T
      Xseti=np.array(np.split(respmat2, DataTensor.shape[2]//TempDimLen, axis=0))
      Yseti=np.zeros((Xseti.shape[0],nclass))
      Yseti[:,idx]=1

      try:
        Xset=np.vstack((Xset, Xseti))
        Yset=np.vstack((Yset, Yseti))
      except:
        Xset=Xseti
        Yset=Yseti
      del(respmat2)
  return Xset, Yset

In [ ]:
try:
  SingleA=np.load('SingleA.npy')
except:
  url1='https://drive.usercontent.google.com/download?id=1BCxLPz0rAnVK-BUl7fYGMHTlsL-sMZoL&export=download&confirm=t&uuid=53a3154a-1a74-44d9-85f1-0202013c5522'
  output1 = '/content/SingleA.npy'
  gdown.download(url1, output1, quiet=True)
  SingleA=np.load('SingleA.npy')

print(SingleA.shape)

TempDimLen=1024
Xset,Yset=DataSplit(SingleA,TempDimLen)
print(Xset.shape,Yset.shape)

In [ ]:
TsetSize=0.2
X_train, X_test, Y_train, Y_test = train_test_split(Xset, Yset, test_size=TsetSize)
del(Xset, Yset)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:

InShape=(1024,30)

def build_conv3D_model(nout=31,fc_dim=8,Shape=InShape):

  model = keras.models.Sequential([keras.layers.Input(shape=Shape),

  keras.layers.Conv1D(filters=2,activation='relu', kernel_size=(51),strides=(3),padding="same", name="Conv1D_1"),
  keras.layers.Conv1D(filters=2,activation='relu', kernel_size=(21),strides=(3),padding="same", name="Conv1D_2"),
  keras.layers.Conv1D(filters=4,activation='relu', kernel_size=(11),strides=(3),padding="same", name="Conv1D_3"),
  keras.layers.Conv1D(filters=4,activation='relu', kernel_size=(7),strides=(2),padding="same", name="Conv1D_4"),
  keras.layers.Conv1D(filters=8,activation='relu', kernel_size=(5),strides=(2),padding="same", name="Conv1D_5"),
  keras.layers.Conv1D(filters=8,activation='relu', kernel_size=(3),strides=(2),padding="same", name="Conv1D_7"),
  keras.layers.Conv1D(filters=8,activation='relu', kernel_size=(3),strides=(2),padding="same", name="Conv1D_8"),

  keras.layers.Flatten(name="encode_flatten"),


  keras.layers.Dense(fc_dim,activation='relu', name="dense_1"),
  keras.layers.Dense(fc_dim,activation='relu', name="dense_2"),

  keras.layers.Dense(nout,activation='softmax', name="dense_3")])
  return model


Model = build_conv3D_model(nout=31,fc_dim=8,Shape=InShape)
Model.summary()

In [ ]:
lrr=5e-3
EPOCHS = 100

optimizer = keras.optimizers.Adam(lrr)#


Model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer=optimizer,metrics='accuracy')#


if not os.path.exists("Models"):
    os.mkdir("Models")
model_checkpoint_callback = ModelCheckpoint(
    filepath = 'Models/BestModel.h5',
    monitor='val_accuracy',#'val_loss',#
    mode='max',#'min',
    save_best_only=True,
    save_weights_only = False)

history = Model.fit(X_train, Y_train, epochs=EPOCHS,batch_size=128, validation_data=( X_test, Y_test),
                     verbose=1,callbacks = [model_checkpoint_callback])#

In [ ]:
n_class=31
bestmodel = load_model('Models/BestModel.h5')
y_pred_probs = bestmodel.predict(X_test)   # Softmax class probabilities from model
y_pred = np.argmax(y_pred_probs, axis = 1)
y_pred_oh = to_categorical(y_pred, n_class)

plot_performance(history,EPOCHS,'Learning Curve',save='/content/Learning Curve')
PlotConfusionMatrix( Y_test, y_pred_probs,figsize=(12,12),save='/content/Confusion MAtrix')

# model scores
scores=score_model(Y_test, y_pred_oh, y_pred_probs,save = '/content/Scores.csv')
# pred_confidence(y_pred_probs)
pd.DataFrame(scores,index=[0])